In [1]:
import geopandas as gpd
import pandas as pd
import itertools

In [2]:
vic_pop = pd.read_csv('../../data/curated/VicPopSA2.csv')
income = pd.read_csv('../../data/landing/income.csv')
sa2Areas = pd.read_csv('../../data/landing/SA2_2021_AUST.csv')

all_properties = pd.read_csv('../../data/landing/Historical Property Data/AllProperties.csv')
flat1 = pd.read_csv('../../data/landing/Historical Property Data/1_Flat.csv')
flat2 = pd.read_csv('../../data/landing/Historical Property Data/2_Flat.csv')
house2 = pd.read_csv('../../data/landing/Historical Property Data/2_House.csv')
house3 = pd.read_csv('../../data/landing/Historical Property Data/3_House.csv')
house4 = pd.read_csv('../../data/landing/Historical Property Data/4_House.csv')

/tmp/ipykernel_137416/3812216178.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  income = pd.read_csv('../../data/landing/income.csv')


In [3]:
# Getting rid of unneccessary rows in sa2Areas

sa2Areas = sa2Areas['SA2_CODE_2021,SA2_NAME_2021,STATE_NAME_2021'.split(',')]

sa2Areas = sa2Areas[sa2Areas['STATE_NAME_2021'] == 'Victoria']

# Convert data types
sa2Areas['SA2_CODE_2021'] = sa2Areas['SA2_CODE_2021'].astype(int)
sa2Areas['SA2_NAME_2021'] = sa2Areas['SA2_NAME_2021'].astype(str)
sa2Areas['STATE_NAME_2021'] = sa2Areas['STATE_NAME_2021'].astype(str)

sa2_code_range = (sa2Areas['SA2_CODE_2021'].min(), sa2Areas['SA2_CODE_2021'].max())


In [4]:
# Filter out non-numeric values in the 'Code' column of the income DataFrame
income = income[pd.to_numeric(income['Code'], errors='coerce').notnull()]
income['Code'] = income['Code'].astype(int)
income = income[income['Code'].between(sa2_code_range[0], sa2_code_range[1])]


income_columns = income.columns.tolist()
columns_to_keep = income_columns[0:3]+income_columns[-9:-1]
income_new = income[columns_to_keep]
income_new.rename(columns={'Label': 'Suburb'})
# MASSIVE DATA LOSS

# Regex
income_new = income_new.replace('-',None)
income_new = income_new.replace('\,', '', regex=True)

# Convert Data Types
float_columns = ['Median equivalised total household income (weekly) ($)','$1-$499 per week (%).1','$500-$999 per week (%).1',
                 '$1000-$1999 per week (%).1','$2000-$2999 per week (%).1','$3000 or more per week (%).1','Nil income (%).1','Partial income stated (%)']
int_columns = ['Year','Code']

for col in float_columns:
    income_new[col] = income_new[col].astype(float)
for col in int_columns:
    income_new[col] = income_new[col].astype(int)

In [5]:
vic_pop = vic_pop[vic_pop['SA2 code'].between(sa2_code_range[0], sa2_code_range[1])]


int_columns = 'SA2 code,2023 no,2022-23 no,Natural increase,Net internal migration,Net overseas migration'.strip().split(',')


for col in vic_pop.columns:
    if str(col) in int_columns:
        vic_pop[col] = vic_pop[col].astype(int)
    elif str(col) == 'SA2 name':
        vic_pop[col] = vic_pop[col].astype(str)
    else:
        vic_pop[col] = vic_pop[col].astype(float)

vic_pop.dtypes

SA2 code                    int64
SA2 name                   object
2023 no                     int64
2022-23 no                  int64
2022-23 %                 float64
Natural increase            int64
Net internal migration      int64
Net overseas migration      int64
Area (km2)                float64
persons/km2               float64
dtype: object

In [6]:
property_data = [all_properties, flat1, flat2, house2, house3, house4]
months = ['Mar', 'Jun', 'Sep', 'Dec']
years = range(2000,2024)


for df in property_data:

    # Rename columns and drop unnecessary rows
    df.rename(columns={'Unnamed: 0': 'Suburb'}, inplace=True)
    months_cycle = itertools.cycle(months)

    i = 1
    year_index = 0
    while i < len(df.columns)-1:
        month = str(next(months_cycle))

        df.rename(columns={df.columns[i]: f'{years[year_index]} {month} Count'}, inplace=True)
        df.rename(columns={df.columns[i+1]: f'{years[year_index]} {month} Median'}, inplace=True)

        year_index = year_index + 1 if month == 'Dec' else year_index
        i += 2
        
    df.drop(df[df['Suburb'] == 'Group Total'].index, inplace=True)

    # Regex
    df.replace('\,', '', regex=True,inplace=True)
    df.replace('-',0,inplace=True)
    df.replace({'\$': ''}, regex=True,inplace=True)
    df.fillna(0,inplace=True)

    # Convert Data Types
    for i in range(len(df.columns)):
        if i == 0:
            df[df.columns[i]] = df[df.columns[i]].astype(str)
        else:
            df[df.columns[i]] = df[df.columns[i]].astype(int)


In [15]:
vic_pop = pd.read_csv('../../data/landing/VicPopHistorical.csv')

vic_pop = vic_pop[vic_pop['SA2 code'].between(sa2_code_range[0], sa2_code_range[1])]
vic_pop = vic_pop[['SA2 code', 'SA2 name', '2001 no', '2002 no', '2003 no', '2004 no', 
    '2005 no', '2006 no', '2007 no', '2008 no', '2009 no', '2010 no', '2011 no', '2012 no',
     '2013 no', '2014 no', '2015 no', '2016 no', '2017 no', '2018 no', '2019 no', '2020 no', '2021 no', '2022 no', '2023 no']]


In [18]:
# Convert data types of vic_pop
vic_pop['SA2 name'] = vic_pop['SA2 name'].astype(str)

for col in vic_pop.columns:
    if col != 'SA2 name':
        vic_pop[col] = vic_pop[col].astype(int)

vic_pop


,SA2 code,SA2 name,2001 no,2002 no,2003 no,2004 no,2005 no,2006 no,2007 no,2008 no,...,2014 no,2015 no,2016 no,2017 no,2018 no,2019 no,2020 no,2021 no,2022 no,2023 no
642,201011001,Alfredton,5756,6092,6293,6480,6648,6761,7034,7272,...,10338,11039,11852,12649,13537,14434,15507,16841,18002,18997
643,201011002,Ballarat,11497,11708,12015,12189,12269,12356,12408,12480,...,12327,12300,12301,12266,12244,12320,12196,12071,11938,11809
644,201011005,Buninyong,5320,5399,5557,5620,5857,6037,6131,6252,...,7082,7191,7311,7409,7418,7458,7377,7229,7247,7323
645,201011006,Delacombe,4154,4225,4371,4465,4704,5041,5206,5349,...,6583,6846,7195,7622,8183,8890,9755,10648,11798,12869
646,201011007,Smythes Creek,3317,3378,3411,3473,3508,3542,3594,3658,...,3945,3966,3990,4004,4042,4112,4152,4211,4223,4268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159,217031476,Otway,3452,3479,3511,3511,3492,3459,3489,3501,...,3519,3538,3556,3635,3710,3802,3911,3979,3974,3983
1160,217041477,Moyne - East,6718,6704,6676,6643,6638,6652,6606,6631,...,6734,6716,6709,6717,6746,6798,6883,6990,7046,7132
1161,217041478,Moyne - West,8317,8387,8450,8487,8517,8601,8694,8792,...,9383,9467,9603,9686,9783,9845,9859,9967,10098,10148
1162,217041479,Warrnambool - North,17053,17449,17726,17937,18172,18528,18877,19107,...,20930,21217,21442,21688,21954,22184,22416,22470,22586,22762


In [19]:
# Save the data
income_new.to_csv('../../data/curated/income_by_suburb.csv', index=False)
vic_pop.to_csv('../../data/curated/VicPopHistoricalSA2.csv', index=False)
sa2Areas.to_csv('../../data/curated/SA2_Victoria.csv', index=False)
all_properties.to_csv('../../data/curated/AllProperties.csv', index=False)
flat1.to_csv('../../data/curated/1_Flat.csv', index=False)
flat2.to_csv('../../data/curated/2_Flat.csv', index=False)
house2.to_csv('../../data/curated/2_House.csv', index=False)
house3.to_csv('../../data/curated/3_House.csv', index=False)
house4.to_csv('../../data/curated/4_House.csv', index = False)